In [25]:
import pandas as pd
import numpy as np
from pathlib import Path

In [26]:
path = Path("data") / "2025 Mercer DA project.xlsx"
df = pd.read_excel(path, sheet_name="mercer benchmark")
print(df.head())

          Geo Comptryx Code              Major Function  \
0  GB: London         RSMP3  R - Research & Development   
1  GB: London         RSMP4  R - Research & Development   
2  GB: London         RSIP1  R - Research & Development   
3  GB: London         RSIP2  R - Research & Development   
4  GB: London         RSIP3  R - Research & Development   

                          Sub Function       Function             Level  \
0  RSM - Mobile Applications Developer  RS - Software  P3 - Proficiency   
1  RSM - Mobile Applications Developer  RS - Software      P4 - Mastery   
2          RSI - User Interface Design  RS - Software        P1 - Entry   
3          RSI - User Interface Design  RS - Software   P2 - Developing   
4          RSI - User Interface Design  RS - Software  P3 - Proficiency   

  Breakout Breakout Value Currency  # Co's  ...  Mkt LTI Total 30th %''ile  \
0  Overall        Overall      GBP     5.0  ...                        0.0   
1  Overall        Overall      GBP    

In [27]:
def standardize_location(Geo):
    if pd.isna(Geo):
        return np.nan
    Geo = str(Geo).strip()

    if "Tokyo" in Geo or Geo.startswith("JP"):
        return np.nan
    # drop Tokyo and Japan-based entries

    if Geo.startswith("GB"):
        if "London" in Geo:
            return "London"
    # standardize UK locations

    if Geo.startswith("SE") or "Sweden" in Geo or "Malmo" in Geo or "Malmö" in Geo:
        return "Malmö"
    # standardize Sweden locations

    if Geo.startswith("PT") or "Lisbon" in Geo or "Portugal" in Geo:
        return "Lisbon"
    # standardize Portugal locations

    if Geo.startswith("US") and ("NY" in Geo or "Tri State" in Geo or "New York" in Geo):
        return "New York"
    # standardize New York locations

    return Geo

df["location_clean"] = df["Geo"].apply(standardize_location)
df = df.dropna(subset=["location_clean"]).copy()



In [28]:
subfunc_title = {
    "RSM - Mobile Applications Developer": "Mobile Engineer",
    "RSG - Software - Generalist": "Full Stack Engineer",
    "RQS - Software Quality Assurance": "QA Engineer",
}

ux_title = {
    "RSI - User Interface Design",
    "RUH - Human Factors Engineering"
}
# standardize job titles

def standardize_job_title(subfunc):
    if pd.isna(subfunc):
        return np.nan
    subfunc = str(subfunc).strip()

    if subfunc in ux_title:
        return "UX Designer"
    return subfunc_title.get(subfunc, np.nan)

df["job_title_clean"] = df["Sub Function"].apply(standardize_job_title)
df = df.dropna(subset=["job_title_clean"]).copy()

In [29]:
level_mapping = {
    "P1 - Entry": "Junior",
    "P2 - Developing": "Mid",
    "P3 - Proficiency": "Senior",
    "P4 - Mastery": "Lead",
    "P5 - Expert": "Principal",
    "M5 - Director": "Head of"
}
# standardize job levels

df["job_level_clean"] = df["Level"].map(level_mapping)
df = df.dropna(subset=["job_level_clean"]).copy()

In [30]:
tx_gbp = {
    "GBP": 1.00,
    "USD": 0.80,
    "EUR": 0.85,
    "SEK": 0.075
}
# currency conversion rates to GBP

def to_gbp(amount, currency):
    if pd.isna(amount) or pd.isna(currency):
        return np.nan
    currency = str(currency).strip().upper()
    rate = tx_gbp.get(currency)
    if rate is None:
        return np.nan
    return amount * rate
# convert salaries to GBP

salary_columns = [
    "Mkt Base Salary 25th %''ile",
    "Mkt Base Salary 50th %''ile",
    "Mkt Base Salary 75th %''ile",
]
for col in salary_columns:
    df[col + "_GBP"] = df.apply(lambda row: to_gbp(row[col], row["Currency"]), axis=1)
# convert all salary columns to GBP

df["mkt_base_avg_GBP"] = df[[col + "_GBP" for col in salary_columns]].mean(axis=1)
# average market base salary in GBP

    

In [31]:
mercer_clean = df[
    [
        "location_clean",
        "job_title_clean",
        "job_level_clean",
        "Mkt Base Salary 25th %''ile_GBP",
        "Mkt Base Salary 50th %''ile_GBP",
        "Mkt Base Salary 75th %''ile_GBP",
        "mkt_base_avg_GBP"
    ]
].rename(columns={
    "Mkt Base Salary 25th %''ile_GBP": "p25_GBP",
    "Mkt Base Salary 50th %''ile_GBP": "p50_GBP",
    "Mkt Base Salary 75th %''ile_GBP": "p75_GBP",
})
# final cleaned DataFrame
print(mercer_clean.head(10))
mercer_clean

  location_clean      job_title_clean job_level_clean      p25_GBP  \
0         London      Mobile Engineer          Senior   65182.0948   
1         London      Mobile Engineer            Lead   87476.0651   
2         London          UX Designer          Junior   42740.2442   
3         London          UX Designer             Mid   42344.5234   
4         London          UX Designer          Senior   60424.4016   
5         London          UX Designer            Lead   75569.3333   
6         London          UX Designer       Principal   90897.6310   
7         London          UX Designer         Head of  113252.9600   
8         London  Full Stack Engineer          Junior   35872.6943   
9         London  Full Stack Engineer             Mid   64366.3255   

       p50_GBP      p75_GBP  mkt_base_avg_GBP  
0   71944.8550   85015.8194      74047.589733  
1   88593.3182   90590.5172      88886.633500  
2   46535.4578   49760.7136      46345.471867  
3   49554.1926   53958.4365      4861

,location_clean,job_title_clean,job_level_clean,p25_GBP,p50_GBP,p75_GBP,mkt_base_avg_GBP
0,London,Mobile Engineer,Senior,65182.09480,71944.85500,85015.81940,74047.589733
1,London,Mobile Engineer,Lead,87476.06510,88593.31820,90590.51720,88886.633500
2,London,UX Designer,Junior,42740.24420,46535.45780,49760.71360,46345.471867
3,London,UX Designer,Mid,42344.52340,49554.19260,53958.43650,48619.050833
4,London,UX Designer,Senior,60424.40160,73127.88960,82628.55890,72060.283367
...,...,...,...,...,...,...,...
67,New York,Full Stack Engineer,Principal,175112.20744,193963.78584,215681.16240,194919.051893
68,New York,Full Stack Engineer,Head of,192021.82400,225458.34520,247376.02936,221618.732853
69,New York,QA Engineer,Mid,77075.69712,82832.56000,91115.81600,83674.691040
70,New York,QA Engineer,Senior,98157.05680,102114.32488,110709.51600,103660.299227


In [32]:
mercer_clean["location_clean"].value_counts()
mercer_clean["job_title_clean"].value_counts()
mercer_clean["job_level_clean"].value_counts()
mercer_clean.isnull().sum()

location_clean      0
job_title_clean     0
job_level_clean     0
p25_GBP             0
p50_GBP             0
p75_GBP             0
mkt_base_avg_GBP    0
dtype: int64

In [46]:
from pathlib import Path
import pandas as pd

PROCESSED_DIR = Path("data/processed")

api_raw = pd.read_csv(PROCESSED_DIR / "api_raw_6levels.csv")
print("api_raw shape:", api_raw.shape)
api_raw.head()

api_raw shape: (138, 13)


,job_title,location,min_salary,max_salary,avg_salary,currency,salary_period,confidence,publisher_name,level_name,years_experience,api_title_requested,api_location_requested
0,Software Developer,"Lisbon, Portugal",1612.5,2679.166667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Junior,0,Software Developer,"Lisbon, Portugal"
1,Software Developer,"Lisbon, Portugal",1612.5,2679.166667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Mid,3,Software Developer,"Lisbon, Portugal"
2,Software Developer,"Lisbon, Portugal",1612.5,2679.166667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Senior,5,Software Developer,"Lisbon, Portugal"
3,Software Developer,"Lisbon, Portugal",1612.5,2679.166667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Lead,7,Software Developer,"Lisbon, Portugal"
4,Software Developer,"Lisbon, Portugal",1612.5,2679.166667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Principal,9,Software Developer,"Lisbon, Portugal"


In [47]:
api_salary = api_raw.dropna(subset=["min_salary", "max_salary", "avg_salary"], how="all").copy()
print("raw:", api_raw.shape, "with salary:", api_salary.shape)
api_salary.head(10)

raw: (138, 13) with salary: (138, 13)


,job_title,location,min_salary,max_salary,avg_salary,currency,salary_period,confidence,publisher_name,level_name,years_experience,api_title_requested,api_location_requested
0,Software Developer,"Lisbon, Portugal",1612.500000,2679.166667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Junior,0,Software Developer,"Lisbon, Portugal"
1,Software Developer,"Lisbon, Portugal",1612.500000,2679.166667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Mid,3,Software Developer,"Lisbon, Portugal"
2,Software Developer,"Lisbon, Portugal",1612.500000,2679.166667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Senior,5,Software Developer,"Lisbon, Portugal"
3,Software Developer,"Lisbon, Portugal",1612.500000,2679.166667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Lead,7,Software Developer,"Lisbon, Portugal"
4,Software Developer,"Lisbon, Portugal",1612.500000,2679.166667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Principal,9,Software Developer,"Lisbon, Portugal"
5,Software Developer,"Lisbon, Portugal",1612.500000,2679.166667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Head of,12,Software Developer,"Lisbon, Portugal"
6,Software Developer,"Lund, Sweden",35167.833333,55491.666667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Junior,0,Software Developer,"Malmo, Sweden"
7,Software Developer,"Lund, Sweden",35167.833333,55491.666667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Mid,3,Software Developer,"Malmo, Sweden"
8,Software Developer,"Lund, Sweden",35167.833333,55491.666667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Senior,5,Software Developer,"Malmo, Sweden"
9,Software Developer,"Lund, Sweden",35167.833333,55491.666667,NaN,NaN,MONTH,VERY_HIGH,Glassdoor,Lead,7,Software Developer,"Malmo, Sweden"


In [48]:
import numpy as np
import pandas as pd

api_clean = api_salary.copy()

def standardize_location_api(loc):
    if pd.isna(loc):
        return None
    s = str(loc).lower()

    if "london" in s or "uk" in s or "gb" in s or "united kingdom" in s:
        return "London"

    if "lisbon" in s or "portugal" in s or s.startswith("pt"):
        return "Portugal"

    if "malmo" in s or "malmö" in s or "lund" in s or "sweden" in s or s.startswith("se"):
        return "Sweden"

    if "new york" in s or "ny" in s or "tri state" in s or "tri-state" in s or "nyc" in s:
        return "New York"

    if "tokyo" in s or "japan" in s:
        return None

    return None  

api_clean["location_clean"] = api_clean["location"].apply(standardize_location_api)
api_clean = api_clean.dropna(subset=["location_clean"])
# Standardize locations

In [49]:
def standardize_job_title_api(title):
    if pd.isna(title):
        return None
    t = str(title).lower()

    if "mobile" in t or "ios" in t or "android" in t:
        return "Mobile Engineer"

    if "ux" in t or "user interface" in t or "human factors" in t or "ui designer" in t:
        return "UX Designer"

    if "full stack" in t or "software - generalist" in t or "software generalist" in t:
        return "Full Stack Engineer"

    if "qa" in t or "quality assurance" in t or "software quality assurance" in t:
        return "QA Engineer"

    return str(title).title()

api_clean["job_title_clean"] = api_clean["job_title"].apply(standardize_job_title_api)

api_clean["job_title_clean"] = api_clean["job_title"].apply(standardize_job_title_api)
api_clean = api_clean.dropna(subset=["job_title_clean"])
# Standardize Job Titles 

In [50]:
for c in ["min_salary", "max_salary", "avg_salary"]:
    api_clean[c] = pd.to_numeric(api_clean[c], errors="coerce")
# Convert to numbers

api_clean["avg_salary"] = api_clean["avg_salary"].fillna(
    (api_clean["min_salary"] + api_clean["max_salary"]) / 2
)
# Add missing average

In [51]:
def annualize(amount, period):
    if pd.isna(amount):
        return np.nan
    p = str(period).lower() if not pd.isna(period) else "year"
    if "month" in p:
        return amount * 12
    if "year" in p:
        return amount
    return amount  

api_clean["min_salary_annual"] = api_clean.apply(lambda r: annualize(r["min_salary"], r["salary_period"]), axis=1)
api_clean["max_salary_annual"] = api_clean.apply(lambda r: annualize(r["max_salary"], r["salary_period"]), axis=1)
api_clean["avg_salary_annual"] = api_clean.apply(lambda r: annualize(r["avg_salary"], r["salary_period"]), axis=1)
# Convert monthly salaries to annual

In [52]:
FX_TO_GBP = {
    "GBP": 1.00,
    "USD": 0.80,
    "EUR": 0.85,
    "SEK": 0.075
}
#Convertion rates

def infer_currency(row):
    if pd.notna(row.get("currency")):
        return str(row["currency"]).upper()
    loc = row["location_clean"]
    if loc == "London":
        return "GBP"
    if loc == "Portugal":  
        return "EUR"
    if loc == "Sweden":
        return "SEK"
    if loc == "New York":
        return "USD"
    return None

api_clean["currency_clean"] = api_clean.apply(infer_currency, axis=1)

def to_gbp(amount, currency):
    if pd.isna(amount) or pd.isna(currency):
        return np.nan
    rate = FX_TO_GBP.get(currency, np.nan)
    return amount * rate

for col in ["min_salary_annual", "max_salary_annual", "avg_salary_annual"]:
    api_clean[col.replace("_annual", "_annual_gbp")] = api_clean.apply(
        lambda r: to_gbp(r[col], r["currency_clean"]), axis=1
    )

api_clean["avg_salary_annual_gbp"] = api_clean["avg_salary_annual_gbp"].fillna(
    (api_clean["min_salary_annual_gbp"] + api_clean["max_salary_annual_gbp"]) / 2
)
 # Currency convertion

In [54]:
api_clean["source_file"] = (
    api_clean["api_title_requested"].str.lower().str.replace(" ", "_", regex=False)
    + "__"
    + api_clean["api_location_requested"].str.lower().str.replace(" ", "_", regex=False)
    + "__"
    + "lvl_" + api_clean["level_name"].str.lower()
)

In [55]:
api_clean = api_clean[[
    "location_clean",
    "job_title_clean",
    "min_salary_annual_gbp",
    "max_salary_annual_gbp",
    "avg_salary_annual_gbp",
    "confidence",
    "publisher_name",
    "source_file"
]]

api_clean.head(10)

,location_clean,job_title_clean,min_salary_annual_gbp,max_salary_annual_gbp,avg_salary_annual_gbp,confidence,publisher_name,source_file
0,Portugal,Software Developer,16447.50,27327.5,21887.500,VERY_HIGH,Glassdoor,"software_developer__lisbon,_portugal__lvl_junior"
1,Portugal,Software Developer,16447.50,27327.5,21887.500,VERY_HIGH,Glassdoor,"software_developer__lisbon,_portugal__lvl_mid"
2,Portugal,Software Developer,16447.50,27327.5,21887.500,VERY_HIGH,Glassdoor,"software_developer__lisbon,_portugal__lvl_senior"
3,Portugal,Software Developer,16447.50,27327.5,21887.500,VERY_HIGH,Glassdoor,"software_developer__lisbon,_portugal__lvl_lead"
4,Portugal,Software Developer,16447.50,27327.5,21887.500,VERY_HIGH,Glassdoor,"software_developer__lisbon,_portugal__lvl_prin..."
5,Portugal,Software Developer,16447.50,27327.5,21887.500,VERY_HIGH,Glassdoor,"software_developer__lisbon,_portugal__lvl_head of"
6,Sweden,Software Developer,31651.05,49942.5,40796.775,VERY_HIGH,Glassdoor,"software_developer__malmo,_sweden__lvl_junior"
7,Sweden,Software Developer,31651.05,49942.5,40796.775,VERY_HIGH,Glassdoor,"software_developer__malmo,_sweden__lvl_mid"
8,Sweden,Software Developer,31651.05,49942.5,40796.775,VERY_HIGH,Glassdoor,"software_developer__malmo,_sweden__lvl_senior"
9,Sweden,Software Developer,31651.05,49942.5,40796.775,VERY_HIGH,Glassdoor,"software_developer__malmo,_sweden__lvl_lead"


In [56]:
from pathlib import Path
PROCESSED_DIR = Path("data/processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

api_clean.to_csv(PROCESSED_DIR / "api_clean.csv", index=False)
print("saved:", PROCESSED_DIR / "api_clean.csv")


saved: data/processed/api_clean.csv
